In [11]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Veri yolu - Klasördeki verilerin kategori alt klasörlerde bulunduğunu varsayalım.
data_dir = 'lang/'

# Veri dönüşümleri (örnek olarak)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Görselleri istediğiniz boyuta dönüştürebilirsiniz
    transforms.ToTensor(),          # Görselleri tensöre dönüştür
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizasyon (örnek)
])

# Veri yükleyici parametreleri
batch_size = 32
num_workers = 4

# Veriyi yükleyin
dataset = datasets.ImageFolder(data_dir, transform=transform)
#dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)


In [12]:
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)  # Giriş boyutunu uygun şekilde ayarlayın
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Modeli oluştur
num_classes = len(dataset.classes)  # Sınıf sayısı otomatik olarak belirlenir
model = CNNModel(num_classes)
model = model.to(device)


In [13]:
# Eğitim parametreleri
learning_rate = 0.001
num_epochs = 10

# Kayıp fonksiyonu ve optimizasyon fonksiyonu
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
correct = 0
total = 0

# Eğitim döngüsü
for epoch in range(num_epochs):
    for images, labels in dataloader:
        # İleri geçiş
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        # Geriye doğru yayılım ve parametre güncelleme
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item()} Accuracy: {accuracy}')


Epoch [1/10] Loss: 0.5321260690689087 Accuracy: 73.02820512820513
Epoch [2/10] Loss: 0.3933107852935791 Accuracy: 81.28205128205128
Epoch [3/10] Loss: 0.1911236196756363 Accuracy: 85.3923076923077
Epoch [4/10] Loss: 0.04932805895805359 Accuracy: 87.99038461538461
Epoch [5/10] Loss: 0.020229006186127663 Accuracy: 89.86410256410257
Epoch [6/10] Loss: 0.2250216156244278 Accuracy: 91.23931623931624
Epoch [7/10] Loss: 0.0021685909014195204 Accuracy: 92.3021978021978
Epoch [8/10] Loss: 0.5739879608154297 Accuracy: 93.11410256410257
Epoch [9/10] Loss: 0.0016689538024365902 Accuracy: 93.73675213675213
Epoch [10/10] Loss: 0.0005498312530107796 Accuracy: 94.28153846153846


In [14]:
# Modeli kaydet
torch.save(model.state_dict(), 'sign.pth')

In [16]:
# Model yapısını oluştur
model = CNNModel(num_classes)  # Model yapısını aynı şekilde oluşturmalısınız

# Kaydedilmiş parametreleri yükle
model.load_state_dict(torch.load('sign.pth'))

# Modeli tahminlerde kullanabilirsiniz
model.eval()  # Modeli tahmin modunda etkinleştirin
# Burada tahminlerinizi yapabilirsiniz


CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=200704, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=26, bias=True)
)

In [17]:
import cv2
from PIL import Image

In [18]:
image_path = 'lang/k/k_1.jpg'  # Test görüntüsünün dosya yolu
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Görüntüyü BGR'den RGB'ye dönüştür
image_pil = Image.fromarray(image)

In [19]:
# Giriş görüntüsünü modele uygun hale getir
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_image = transform(image_pil)
input_image = input_image.unsqueeze(0)  # Batch boyutunu ekleyin (1 görüntü için)

In [20]:
with torch.no_grad():
    output = model(input_image)

# Sınıflandırma sonuçlarını al
_, predicted = torch.max(output, 1)

# Sınıf tahminini bul
class_index = predicted.item()
class_label = dataset.classes[class_index]  # 'dataset' veri kümesinin sınıflarını içerdiğini varsayalım

print(f'Tahmin edilen sınıf: {class_label}')

Tahmin edilen sınıf: k
